In [18]:
import pandas as pd 
import os 
from 
# from openai.embeddings_utils import get_embedding
import tiktoken

from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import podcast_scraper

In [21]:
podcast = podcast_scraper.load_podcast_json_to_dataframe('../data/podcast_data.json')
podcast['combined_info'] = podcast.apply(lambda row: f"Title: {row['podcast']}. Overview: {row['description']} Genres: {row['type']}", axis=1)



In [23]:
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

encoding = tiktoken.get_encoding(embedding_encoding)

# omit descriptions that are too long to embed
podcast["n_tokens"] = podcast.combined_info.apply(lambda x: len(encoding.encode(x)))
podcast = podcast[podcast.n_tokens <= max_tokens]

#### Storing and documenting in a vector store 
Use Pinecone as a vectorstore and OpenAI embeddings as embedding model

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['API_KEY']) 

pinecone.init(
api_key= os.environ['PINECONE_KEY'], environment= 'us-west1-gcp-free')
index_name = pinecone.Index('index-podcast')




#### Retrieval and building retrieval chain
Use Langchain retriever with Langchain ConversationalRetrieval chain

In [ ]:
vectordb = Pinecone.from_documents(texts, embeddings, index_name='index-1')
retriever = vectordb.as_retriever()

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

In [ ]:
query = "I'm looking for a podcast similar to Conan O'Brian Needs a Friday. What would you suggest to me"
result = chain.run({'question': query})

#### Prompt engineering 
Pass custom prompt with information about users 


In [ ]:
template = """
You're a podcast recommender system that helps users to find the right podcast that match their interest. 
Use the following pieces of context to answer the question at the end.
For each question, use the context and input provided by the user

{context}
"""
user_info = """ The following are user input provided by the users 

Age: {age}
Gender: {gender}
Profession: {profession}
Topics of interest: {interest}

"""

template_suffix = """"""